In [1]:
import requests
import json

In [3]:
data = {
    "prompt": "[INST] Who is Elon Musk? [/INST]",
    "sampling_params": {
        "temperature": 0.9,
    },
}

url = "http://127.0.0.1:8000/llm/generate"
response = requests.post(url, json=data)
# response.json()

In [5]:
# whisper transcribe stream example
import requests, json, pdb

data = {
    "video": {"path": "/workspaces/aana_sdk/aana/tests/files/videos/physicsworks.webm"},
}

url = "http://127.0.0.1:8001/video/transcribe"

response = requests.post(url, data={"body": json.dumps(data)}, stream=True)
for chunk in response.iter_content(chunk_size=None):
    json_data = json.loads(chunk)

    if "error" in json_data:
        print(json_data["error"])
        print(json_data["message"])
        print(json_data["stacktrace"])
        break

    segments = json_data.get("segments")
    if segments:
        print(segments[0]["text"])
    else:
        print("\n")
        print("===================")
        print("End of Transcription")

 Now I want to return to the conservation of mechanical energy.
 I have here a pendulum.
 I have an object that weighs 15 kilograms
 and I can lift it up one meter, which I have done now.
 That means I've done work. MGH is the work I have done,
 believe me.
 I've increased the potential energy of this object.
 Then that will be converted to kinetic energy.
 If I would let it swing from one meter height
 and you would be there and it would hit you, you'd be dead.
 150 joules is enough to kill you.
 They use these devices. They're called a wrecker ball.
 They use them to demolish buildings.
 You lift up a very heavy object, even heavier than this,
 you let it go, you swing it,
 thereby converting gravitational potential energy
 into kinetic energy,
 and that way you can demolish a building.
 You just let it hit and it breaks a building.
 And that's the whole idea of wrecking.
 So you are using, then, the conversion
 of gravitational potential energy to kinetic energy.
 Now, I am such a s

In [11]:
import requests, json

# whisper batched transcribe stream example
#  Tested with: mp4, mxf, wav, mpeg, mov
#  MP3, AAC, m4a, ts
# Tested with video, audio and video without audio channel
data = {
    "video": {
        "path": "/workspaces/aana_sdk/aana/tests/files/videos/physicsworks.webm",
    },
}

url = "http://127.0.0.1:8001/video/transcribe_batch"

response = requests.post(url, data={"body": json.dumps(data)}, stream=True)
for chunk in response.iter_content(chunk_size=None):
    json_data = json.loads(chunk)
    # print(type(json_data))
    # print(json_data.keys())
    if "error" in json_data:
        print(json_data["error"])
        print(json_data["message"])
        print(json_data["stacktrace"])
        break

    segments = json_data.get("segments")
    transcription = json_data.get("transcription")

    if segments:
        # print(segments[0]["text"])
        print(transcription["text"])
    else:
        if segments == []:  # Empty segments for no-audio case
            continue
        print("===================")
        print("End of Transcription")

# print(json_data)  # To get transcription_id as well

 Now I want to return to the conservation of mechanical energy. I have here a pendulum. I have an object that weighs 15 kilograms and I can lift it up one meter, which I have done now. That means I've done work. mgh is the work I have done, believe me. I've increased the potential energy of this object. 15 times 10 is about 150 joules.
 then that will be converted to kinetic energy. If I would let it swing from one meter height and you would be there and it would hit you, you'd be dead. 150 joules is enough to kill you. They use these devices. They're called a racquetball. They use them to demolish buildings. You lift up a very heavy object, even heavier than this
 And then you let it go, you swing it, thereby converting gravitational potential energy into kinetic energy, and that way you can demolish a building. You just let it hit... and it breaks a building. And that's the whole idea of wrecking. So you are using, then, the conversion of gravitational potential energy to kinetic ene

In [2]:
# transcribe sequential benchmarking from /nas dataset
import os, requests, json
import time

# documentary videos
url = "http://127.0.0.1:8001/video/transcribe_batch"
base_dir = "/nas/jilt/datasets/Documentaries/Crawler/all_videos"

paths = [
    os.path.join(base_dir, pth) for pth in os.listdir(base_dir) if pth.endswith(".mp4")
]
difference = []
for i in range(10):
    start_time = time.time()
    for path in paths:
        data = {
            "video": {"path": path},
        }
        response = requests.post(url, data={"body": json.dumps(data)}, stream=True)

        for chunk in response.iter_content(chunk_size=None):
            json_data = json.loads(chunk)

            if "error" in json_data:
                print(json_data["error"])
                print(json_data["message"])
                print(json_data["stacktrace"])
                break

            segments = json_data.get("segments")

    differ = time.time() - start_time
    difference.append(differ)
print(difference)

[100.40857911109924, 92.53680682182312, 93.19299221038818, 92.7681987285614, 92.4898133277893, 91.91909074783325, 92.23066234588623, 91.05932140350342, 91.68245553970337, 94.19734644889832]


In [27]:
# transcribe threading benchmarking based on /nas dataset with 2 whisper replicas on 2 small ada GPUs
import os, requests, json
import time
import threading
from concurrent.futures import ThreadPoolExecutor

# documentary videos
url = "http://127.0.0.1:8001/video/transcribe_batch"
base_dir = "/nas/jilt/datasets/Documentaries/Crawler/all_videos"

paths = [
    os.path.join(base_dir, pth) for pth in os.listdir(base_dir) if pth.endswith(".mp4")
]

difference = []


def predict_on_video(path):
    data = {
        "video": {"path": path},
    }
    response = requests.post(url, data={"body": json.dumps(data)}, stream=True)
    op_text = []
    for chunk in response.iter_content(chunk_size=None):
        json_data = json.loads(chunk)
        if "error" in json_data:
            print(json_data["error"])
            print(json_data["message"])
            print(json_data["stacktrace"])
            break

        segments = json_data.get("segments")
        if segments:
            op_text.append(segments[0]["text"])
        else:
            return op_text


# now do it for 20 iterations
difference = []
for i in range(10):
    start_time = time.time()
    with ThreadPoolExecutor(max_workers=10) as executor:

        def worker_func(path):
            return predict_on_video(path)

        results = list(
            zip(paths, executor.map(worker_func, paths))
        )  # should get a list of lists.
    differ = time.time() - start_time
    difference.append(differ)
print(difference)

[45.32583928108215, 47.194429874420166, 44.753589391708374, 45.29305362701416, 44.94037747383118, 44.15987682342529, 43.98680377006531, 49.56646513938904, 43.43703770637512, 44.23016595840454]
